## Assignment - A3.part2
### Devin Hall, Tyler Birnie
### Due: 3-25-22 @ 11:59pm
Some pre-requisites we will need for this project

### Question 1: Pokemon Data
Load in the data, look at the distribution of Pokemon skills (data features) we will use for dimenionality reduction. "HP, Attack, Defense, SpAtk, SpDef, and Speed"

#### Question 1A:
create a violin plot showing the distribution of these variables
Helpful functions: python- seaborn package; violinplot


#### Question 1B: 
The six features (Pokemon Skills) have different ranges, therefore we should scale the data before considering PCA. Standardize the data (center the data- subtract the mean, and divide by the standard deviation). 

_"Note, we only need to scale the data for the 6 variables under consideration"_
Helpful functions: python- StandardScaler from sklearn.preprocessing

In [4]:
## use StandardScaler from sklearn.preprocessing to standardize the data 
# of the 6 variables of interest

#### Quesetion 1C:
Preform principle components analysis (PCA) on the scaled Pokemon skills data. 
Helpful funcitons: PCA function sklearn.decomposition

#### Question 1D:
Plot the transformed data in the space defined by the first two principal components.  

[This should be a scatter plot]
helpful functions: python- matplotlib, scatter

#### Question 1E:
Explore the amount of variance explained by each principle component direction. 
First, plot the proportion of variance explained (y-axis) vs. the different principle components. 
Second, plot the cummulative proportion of variance explained y the principle components. Make sure this plot has y-limits from 0 - 1.
helpful functions: python- matplotlib, scatter or plot

### Question 2: College Data
The set of colleges considered are the 21 top colleges based on earning potential of undergraduates with computer science degrees: "college_data.csv" (Sources: 2014- Payscale.com and the National Center for Education Statistics, NCES)
[load the data from college_data.csv]

#### Question 2A:
Preform principal component analysis on the college data (make sure to preprare the data in any way needed)
Helpful functions: python- PCA from sklearn.decomposition

#### Questiton 2B:
Plot the dat in the space defined by the first two principal components (labeling each point with the school it represents)

#### Quesiton 2C:
Plot the amount of cumulative variance explained.
How many principal components should be used for any further analysis to be done on the data?

### Question 3: Text Classification
For this question you will be considering text classificatoin using two different Naive Bayes models. These approaches will be discussed in class and referenced from the following book.

Manning, C., Raghavan, P., Schutze, H. Introduction to Information Retrieval, Cambridge University Press, 2008
http://nlp.stanford.edu/IR-book/

In particular, look at Chapter 13 http://nlp.stanford.edu/IR-book/html/htmledition/text-classification-and-naive-bayes-1.html

You will be using data from the Presidential State of the Union Addresses available as a zip archive. The speeches are available in text files sorted by year, e.g., a1.txt, ..., a231.txt. The text files are formated such that there is one word per line and most punctuation has been removed. Note, there are still hyphens or dashes left in the text files and there may be some errors in splitting of words.

#### Quesiton 3A:
Load the addresses. You will need to create a vector listing the party affiliation of each president to match their speech, you may use the file "party.txt" to help with this classification.


#### Question 3B:
Remove _stopwords_ from consideration for the method. The stopwords are availabe at "stopwords.txt" or found in various packages nltk and 'tm.

#### Question 3C:
Predict the part affiliations (Democrat / Republican) for the following speeches:
- Donald Trump, 2017
- Barrack Obama, 2014
- George W. Bush, 2006
- William Clinton, 1995
- John F. Kennedy, 1962

The training set will be the rmaining speeches that can be associated with the Democratic or Republican presidents(note, you will not need all the addresses, but they were included here for completness of the data). You will need to complete the following steps:


##### Question 3Ci:
Create a term-document matrix, T D for this set of speeches. Restrict this matrix to the 3000 most frequently used words over all the speeches (not including the stopwords already removed). Show the first 10 rows and 5 columns.

##### Question 3Cii:
or the 5 speeches listed above determine the party affiliations of the
president. Calculate and report P (C = Dems |X) and P (C = Reb |X) under the Bernoulli model of Na ̈ıve Bayes. 
Helpful functions: sklearn.naive bayes.BernoulliNB in Python

##### Question 3Ciii:
For the 5 speeches listed above determine the party affiliations of the
president. Calculate and report P (C = Dems |X) and P (C = Reb |X) under the Multinomial model of Na ̈ıve Bayes.
Helpful functions: sklearn.naive bayes.MultinomialNB in Python 

##### Question 3Civ:
Create a plot showing the top 20 most frequenctly used words across all the Republican and Decmocrate presidents

##### Question 3Cv:
Create a plot showing the top 10 most frequently used words for Republicans and Democrats in a small multiple or faceted plot

### Question 4: Text Classification - Bonus
Repeat the analysis from above, but implement the Bernoulli Naive Bayes and Multinomial Naive Bayes models yourself. Follow the pseudocode given in the the IR book. Note,for this question you will not use the standard Na ̈ıve Bayes package, library or function.
This means you can’t use sklearn.naive bayes.BernoulliNB in Python
You can’t use sklearn.naive bayes.MultinomialNB in Python


#### Question 4A: 
For the 5 speeches listed above determine the party affiliations of
the president. Calculate and report P (C = Dems |X) and P (C = Reb |X) under the Bernoulli model of Na ̈ıve Bayes. In order to avoid underflow errors, use the log probabilities as discussed in class.

#### Question 4B: 
For the 5 speeches listed above determine the party affiliations of the
president. Calculate and report P (C = Dems |X) and P (C = Reb |X) under the Multinomial model of Na ̈ıve Bayes.

Hint: A strong suggestion for this question is to first create and test your code on a small document set, e.g., the example in the IR-book. Once you have that working correctly, then run on the SOTU addresses